# Team SML

In [136]:
import pandas as pd
import geopandas as gpd
import geopy.distance
import math
import requests
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression


In [5]:


url = 'https://raw.githubusercontent.com/cividi/st-gallen-urban-indicators/main/data/price-monitoring/price-monitoring-extended.csv'
df = pd.read_csv(url, index_col=0)
print(df.head(5))

mycols = ['rent','lat', 'lng', 'LR_DAY', 'LR_NIGHT']

pipe = Pipeline([
('selector', ("selector", ColumnTransformer([
        ("selector", "passthrough", mycols)
    ], remainder="drop"))),
    ('regression', LinearRegression())
])

pipeline.fit(X=df.iloc[:, 0:5], y=df['rent'])

                                                   object_ref category  \
id                                                                       
108761792                                         hgok2876852     rent   
2147994348  44.53.3003.499622.e75b7976-c57b-11e9-9cb0-a4bf...     rent   
3000166218                                    21301.01.420030     rent   
3000251780                                        hgoe3023792     rent   
3000402506                                      03159.01.0132     rent   

                           date  flatType floor  rent  rent_add  rent_net  \
id                                                                          
108761792   2021-10-12 19:06:18       NaN     4  1745     200.0    1545.0   
2147994348  2021-10-12 19:08:41       NaN     3  1720       NaN       NaN   
3000166218  2021-10-12 20:47:01       NaN     2  1150       NaN       NaN   
3000251780  2021-10-13 07:27:31       NaN     2  1390       NaN       NaN   
3000402506  2021-10

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' '('selector', ColumnTransformer(transformers=[('selector', 'passthrough',
                                 ['rent', 'lat', 'lng', 'LR_DAY', 'LR_NIGHT'])]))' (type <class 'tuple'>) doesn't

In [ ]:
urlNew = 'https://raw.githubusercontent.com/cividi/st-gallen-urban-indicators/main/data/price-monitoring/price-monitoring-extended.csv'

df_rentalobjects = pd.read_csv(urlNew, index_col=0)
df_supermarkets = gpd.read_file('./mapdata/supermarkets.geojson')
df_leisure_places = gpd.read_file('./mapdata/leisure_places.geojson')
df_othershops = gpd.read_file('./mapdata/othershops.geojson')
df_public_transport_stations = gpd.read_file('./mapdata/public_transport_stations.geojson')
df_bus_stations = gpd.read_file('./mapdata/bus_stops.geojson')
df_rentalobjects['supermarketsnearby'] = 0
df_rentalobjects['leisurelocationsnearby'] = 0
df_rentalobjects['othershopsnearby'] = 0
df_rentalobjects['smallestdistancetonextoev'] = None
df_rentalobjects['distancetomainstation'] = None
df_rentalobjects['altitude'] = None
df_rentalobjects['distancetonightclubs'] = None
df_rentalobjects = df_rentalobjects.reset_index()
df_supermarkets = df_supermarkets.reset_index()
df_leisure_places = df_leisure_places.reset_index()
df_othershops = df_othershops.reset_index()
df_public_transport_stations = df_public_transport_stations.reset_index()
df_bus_stations = df_bus_stations.reset_index()

for index, row_rentalobject in df_rentalobjects.reset_index().iterrows():
    count_supermarkets_nearby = 0
    count_leisurelocations_nearby = 0
    count_othershops_nearby = 0
    smallest_distance_to_next_oev_station = None
    distance_to_main_station = None
    distance_to_nightclubs = None
    altitude = None
    if not math.isnan(row_rentalobject['lng']) and not math.isnan(row_rentalobject['lat']):
        coords_rentalobject = (row_rentalobject['lat'], row_rentalobject['lng'])
        for index2, row_supermarket in df_supermarkets.iterrows():
            geometry_supermarket = row_supermarket['geometry']
            coords_supermarket = (geometry_supermarket.y, geometry_supermarket.x)
            if not math.isnan(geometry_supermarket.y) and not math.isnan(geometry_supermarket.x):
                if geopy.distance.distance(coords_rentalobject, coords_supermarket).m <= 500:
                    count_supermarkets_nearby = count_supermarkets_nearby + 1
                    
        for index3, row_leisurelocations in df_leisure_places.iterrows():
            geometry_leisurelocation = row_leisurelocations['geometry']
            coords_leisurelocation = (geometry_leisurelocation.y, geometry_leisurelocation.x)
            if not math.isnan(geometry_leisurelocation.y) and not math.isnan(geometry_leisurelocation.x):
                if geopy.distance.distance(coords_rentalobject, coords_leisurelocation).m <= 500:
                    count_leisurelocations_nearby = count_leisurelocations_nearby + 1
                    
        for index4, row_othershops in df_othershops.iterrows():
            geometry_othershops = row_othershops['geometry']
            coords_othershops = (geometry_othershops.y, geometry_othershops.x)
            if not math.isnan(geometry_othershops.y) and not math.isnan(geometry_othershops.x):
                if geopy.distance.distance(coords_rentalobject, coords_othershops).m <= 500:
                    count_othershops_nearby = count_othershops_nearby + 1
                    
        for index5, row_public_transport_stations in df_public_transport_stations.iterrows():
            geometry_public_transport_stations = row_public_transport_stations['geometry']
            coords_public_transport_stations = (geometry_public_transport_stations.y, geometry_public_transport_stations.x)
            if not math.isnan(geometry_public_transport_stations.y) and not math.isnan(geometry_public_transport_stations.x):
                current_distance_to_next_oev = geopy.distance.distance(coords_rentalobject, coords_public_transport_stations).m
                if  smallest_distance_to_next_oev_station == None or current_distance_to_next_oev < smallest_distance_to_next_oev_station:
                    smallest_distance_to_next_oev_station = current_distance_to_next_oev
        
        coords_main_station = (47.42299, 9.36951)            
        distance_to_main_station = geopy.distance.distance(coords_rentalobject, coords_main_station).m
        coords_nightclubs = (47.42614, 9.37737) 
        distance_to_nightclubs = geopy.distance.distance(coords_rentalobject, coords_nightclubs).m
        requesturl = 'https://api.open-elevation.com/api/v1/lookup?locations=' + str(row_rentalobject['lat']) + ',' + str(row_rentalobject['lng'])
        r = requests.get(requesturl)
        altitude = r.json()['results'][0]['elevation']
        #for index6, row_bus_stations in df_bus_stations.iterrows():
         #   geometry_bus_stations = row_bus_stations['geometry']
          #  coords_bus_stations = (geometry_bus_stations.y, geometry_bus_stations.x)
           # if not math.isnan(geometry_bus_stations.y) and not math.isnan(geometry_bus_stations.x):
            #    current_distance_to_next_oev = geopy.distance.distance(coords_rentalobject, coords_bus_stations).m
             #   if  smallest_distance_to_next_oev_station == None or current_distance_to_next_oev < smallest_distance_to_next_oev_station:
              #      smallest_distance_to_next_oev_station = current_distance_to_next_oev  
    df_rentalobjects.at[index, ['supermarketsnearby', 'leisurelocationsnearby', 'othershopsnearby', 'smallestdistancetonextoev', 'distancetomainstation', 'altitude', 'distancetonightclubs']] = count_supermarkets_nearby, count_leisurelocations_nearby, count_othershops_nearby, smallest_distance_to_next_oev_station, distance_to_main_station, altitude, distance_to_nightclubs
df_rentalobjects.to_csv("./data/price-monitoring-calculated.csv", index=False)
print('done')
        
        



